# Como analisar os dados de rentabilidade?

Desafio:
Construir um código que faça um ranking dos melhores fundos em três períodos de rentabilidade diferentes. Iremos selecionar os 10 melhores posicionados no quesito rentabilidade no curto e no longo prazo.

Passo a passo:
Passo 1 - Puxar as tabelas igual na aula 2.

Passo 2 - Escolher quais períodos de rentabilidade iremos analisar.

Passo 3 - Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora.

Passo 4 - Transformar as rentabilidades em números decimais, ao invés de string.

Passo 5 - Juntar as tabelas, igual na aula 2.

Passo 6 - Filtrar os ETFs alavancados.

Passo 7 - Fazer os rankings de rentabilidade por período e soma-los.

Passo 8 - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

# Passo 1: Realizar o webscraping

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

In [4]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

driver.get('https://www.etf.com/etfanalytics/etf-finder')

time.sleep(5)

botao_100 = driver.find_element('xpath',
                                '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span')

driver.execute_script('arguments[0].click();', botao_100)
#botao_100.click()

numero_paginas_element = driver.find_element('xpath', '//*[@id="totalPages"]')
numero_paginas_text = numero_paginas_element.text
match = re.search(r'(\d+)', numero_paginas_text)

elemento = driver.find_element('xpath', '//*[@id="finderTable"]')
html_tabela = elemento.get_attribute('outerHTML')
tabela = pd.read_html(str(html_tabela))[0]

lista_tabela_por_pagina = []

elemento = driver.find_element('xpath', '//*[@id="finderTable"]')


for pagina in range(1, numero_paginas + 1):
    
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_avancar_pagina = driver.find_element('xpath', '//*[@id="nextPage"]')
    
    driver.execute_script('arguments[0].click();', botao_avancar_pagina)
    

tabela_cadastro_etfs = pd.concat(lista_tabela_por_pagina)

formulario_de_voltar_pagina = driver.find_element('xpath', '//*[@id="goToPage"]')

formulario_de_voltar_pagina.clear()
formulario_de_voltar_pagina.send_keys('1')
formulario_de_voltar_pagina.send_keys(u'\ue007')

botao_mudar_pra_performance = driver.find_element('xpath', '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span')

driver.execute_script('arguments[0].click();', botao_mudar_pra_performance)

lista_tabela_por_pagina = []

elemento = driver.find_element('xpath', '//*[@id="finderTable"]')


for pagina in range(1, numero_paginas + 1):
    
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_avancar_pagina = driver.find_element('xpath', '//*[@id="nextPage"]')
    
    driver.execute_script('arguments[0].click();', botao_avancar_pagina)
    

tabela_rentabilidade_etfs = pd.concat(lista_tabela_por_pagina)

driver.quit()

In [5]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.set_index('Ticker')

tabela_cadastro_etfs = tabela_cadastro_etfs.set_index('Ticker')

# Passo 2: Escolher quais períodos de rentabilidade iremos analisar.

In [7]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs[['1 Year', '3 Years', '5 Years']]

# Passo 3: Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora.

In [10]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.replace('--', pd.NA)

tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.dropna()

# Passo 4: Transformar as rentabilidades em números decimais

In [11]:
for coluna in tabela_rentabilidade_etfs.columns:
    tabela_rentabilidade_etfs[coluna] = tabela_rentabilidade_etfs[coluna].str.rstrip('%').astype(float)/100

# Passo 5: Juntar as tabelas

In [13]:
base_final = tabela_cadastro_etfs.join(tabela_rentabilidade_etfs, how = "inner")

# Passo 6: Filtrar os ETFs alavancados.

In [15]:
base_final = base_final[~base_final['Segment'].str.contains("Leveraged")]

# Passo 7: Fazer os rankings de rentabilidade por período e soma-los.

In [17]:
base_final['rank_1_anos'] = base_final['1 Year'].rank(ascending = False)
base_final['rank_3_anos'] = base_final['3 Years'].rank(ascending = False)
base_final['rank_5_anos'] = base_final['5 Years'].rank(ascending = False)
base_final['rank_final'] = (base_final['rank_1_anos'] + 
                                  base_final['rank_3_anos'] + 
                                  base_final['rank_5_anos'])